In [1]:
print("ok")

ok


In [30]:
%pwd

'c:\\Users\\manum\\Desktop\\Medical Chat AI\\End-to-end-medical-Chatbot-AI\\research'

In [31]:
import os
os.chdir("../")

In [32]:
%pwd

'c:\\Users\\manum\\Desktop\\Medical Chat AI\\End-to-end-medical-Chatbot-AI'

In [33]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [34]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [35]:
extracted_data=load_pdf_file(data='Data/')

In [37]:
# extracted_data

In [38]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [39]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5694


In [41]:
# text_chunks

In [52]:
from langchain.embeddings import HuggingFaceEmbeddings

In [53]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [55]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Model loaded successfully!")


c:\Users\manum\Anaconda3\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\manum\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Model loaded successfully!


In [56]:
embeddings = download_hugging_face_embeddings()

In [57]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [79]:
from dotenv import load_dotenv
load_dotenv()

True

In [76]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [90]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [62]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [63]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [64]:
docsearch

In [65]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [66]:
retrieved_docs = retriever.invoke("What is Acne?")

In [67]:
retrieved_docs

[Document(id='9c95a03d-875e-4807-b731-76a8b50a10c1', metadata={'page': 4.0, 'page_label': '5', 'source': 'Data\\Medical encyclopedia.pdf'}, page_content='A\nAbscess\nAcidophilus\nAcne\nAconite\nAcupressure\nAcupuncture\nAdemetionine\nAdie’s pupil\nAfrican pygeum\nAgastache\nAging\nAIDS\nAlcoholism\nAlexander technique\nAlfalfa\nAlisma\nAllergies\nAllium cepa\nAloe\nAlpha-hydroxy\nAlzheimer’s disease\nAmino acids\nAndrographis\nAndrostenedione\nAnemarrhena\nAnemia\nAngelica root\nAngina\nAnise\nAnkylosing spondylitis\nAnorexia nervosa\nAnthroposophical medicine\nAntioxidants\nAnxiety\nBates method\nBayberry\nBedsores\nBedwetting\nBee pollen\nBehavioral optometry\nBehavioral therapy'),
 Document(id='25729cd6-d2a9-4fc0-b14d-dacb67e2f5f4', metadata={'page': 99.0, 'page_label': '100', 'source': 'Data\\Medical encyclopedia.pdf'}, page_content='Atopy —A group of diseases, including eczema,\nthat develop in people with an inherited tendency\nto develop immediate antibodies to common en-\nviron

In [81]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [97]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [98]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])